In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

csv.field_size_limit(1000000000000000000)

131072

In [2]:
# import FOITEXT and MASDEV files
left = pd.read_pickle("./3_FOITEXT_File.pkl")
right = pd.read_pickle("./4_MASDEV_Censored.pkl")

In [3]:
print("shape of FOITEXT: " + str(left.shape))
print("shape of MASTER_DEVICE: " + str(right.shape))

shape of FOITEXT: (10419137, 7)
shape of MASTER_DEVICE: (94786, 33)


In [4]:
df = pd.merge(left, right, on="MDR_REPORT_KEY", how="inner", indicator=True)
df["_merge"].value_counts()

both          91259
left_only         0
right_only        0
Name: _merge, dtype: int64

In [5]:
df.head()

,MDR_REPORT_KEY,MDR_TEXT_KEY,TEXT_TYPE_CODE,PATIENT_SEQUENCE_NUMBER,DATE_REPORT,FOI_TEXT,textLen,REPORT_NUMBER,REPORT_SOURCE_CODE,MANUFACTURER_LINK_FLAG_,...,MODEL_NUMBER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,year,STD_MANUFACTURER,STD_DEVICE,RECALL_CLASS,CENSORED,_merge
0,263018,176382,D,1,NaN,6963 APPARENT LEAD FRACTURE. 6966 TESTED OUT O...,NaN,2182208-2000-00046,M,Y,...,6966,LWS,4 YR,Y,2000,MEDTRONIC,MEDTRONIC_TRANSVENE,0,2000-01-10,both
1,263038,15441668,D,1,NaN,NO DEVICE INTERROGATION POSSIBLE. PT ALERT WAS...,NaN,2182208-2000-00036,M,Y,...,7273,LWS,10 MO,R,2000,MEDTRONIC,MEDTRONIC_GEM_DR,1,2000-01-10,both
2,263071,152400,D,1,NaN,ICD TESTED OUT OF SPECIFICATIONS DURING MFGR'S...,NaN,2647346-2000-00025,M,Y,...,7221B,LWS,24 MO,Y,2000,MEDTRONIC,MEDTRONIC_MICRO_JEWEL,1,2000-01-10,both
3,263075,176391,D,1,NaN,"DISSATISFIED WITH RESPONSE/REPAIR, LONG CHARGE...",NaN,2182208-2000-00030,M,Y,...,7271,LWS,0 DAY,R,2000,MEDTRONIC,MEDTRONIC_GEM_DR,1,2000-01-10,both
4,263095,176394,D,1,NaN,ALLEGED MALFUNCTION/DEFECTIVE/BAD/FAILURE,NaN,2182208-2000-00024,M,Y,...,6936,LWS,4 YR,Y,2000,MEDTRONIC,MEDTRONIC_TRANSVENE,0,2000-01-10,both


In [6]:
# make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.
def cleanTextRound1(text):
    text = text.upper()
    text = re.sub('\[.*?\‘’“”…]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    return text

round1 = lambda x: cleanTextRound1(x)


data_clean = pd.DataFrame(df.FOI_TEXT.apply(round1))
data_clean.head(5)

,FOI_TEXT
0,APPARENT LEAD FRACTURE TESTED OUT OF SPECIFI...
1,NO DEVICE INTERROGATION POSSIBLE PT ALERT WAS ...
2,ICD TESTED OUT OF SPECIFICATIONS DURING MFGRS ...
3,DISSATISFIED WITH RESPONSEREPAIR LONG CHARGE T...
4,ALLEGED MALFUNCTIONDEFECTIVEBADFAILURE


In [7]:
# checking the first 100 reports in the df
n=0

for entry in data_clean["FOI_TEXT"]:
    if n < 100:
        print("[" + str(n) + "]\n" + entry + "\n")
        n += 1    

[0]
 APPARENT LEAD FRACTURE  TESTED OUT OF SPECIFICATIONS DURING ANALYSIS

[1]
NO DEVICE INTERROGATION POSSIBLE PT ALERT WAS ACTIVATED FOR EXCESSIVE CHARGE TIME LEAD IMPEDANCE AND BATTERY VOLTAGE

[2]
ICD TESTED OUT OF SPECIFICATIONS DURING MFGRS ANALYSIS

[3]
DISSATISFIED WITH RESPONSEREPAIR LONG CHARGE TIME NOT ERI TACHY

[4]
ALLEGED MALFUNCTIONDEFECTIVEBADFAILURE

[5]
OVERSENSING FROM LEAD FRACTURE CAUSED PT TO RECEIVE UNNECESSARY ICD DISCHARGES

[6]
ATTEMPTED IMPLANT  TESTED OUT OF SPECIFICATIONS DURING MFGRS ANALYSIS

[7]
TESTED OUT OF SPECIFICATIONS DURING ANALYSIS

[8]
FRACTURED ELECTRODE COIL DEFIB COILPATCH

[9]
DEVICE INOPERABLE WHEN THERAPY ATTEMPTED DUE TO APPARENT FRACTURED NONMEDTRONIC LEAD  TESTED OUT OF SPECIFICATIONS DURING ANALYSIS

[10]
TESTED OUT OF SPECIFICATIOSN DURING MFGRS ANALYSIS  DEVICE NOTED TO BE OK AT EXPLANT RETURNED  MONTHS FOLLOWING EXPLANT

[11]
  UNWANTEDUNEXPECTED SHOCK INTERFERENCENOISE INAPPROPRIATE THERAPY      TESTED OUT OF SPECIFICATIONS DURING 

In [8]:
# replace manufacturer identifiers
# spaces were not used in regexs as words may not be spaced
def cleanTextRound2(text):
    text = re.sub('''SJM|TELECTRONICS PACING SYSTEMS INC|RICE CREEK|MPROC|MDT|EOC HEERLEN DISTRIBUTION CTR
                        |BOSTON SCIENTIFIC|GUIDANT|CPI|CARDIAC PACEMAKER|CLONMEL|CAMERON HEALTH|HISTORICAL PUERTO RICO
                        |EXTERNAL MANUFACTURERPCI PLANT|SORI|ANGEION|MPRI|IPG MFG|ST JUDE|ST JUDE MEDICAL|MEDTRONIC 
                        |BOSTON SCIENTIFIC|BIOTRONIK|SORIN''', " MFGID ", text)
    return text

round2 = lambda x: cleanTextRound2(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round2))
data_clean.head(5)

,FOI_TEXT
0,APPARENT LEAD FRACTURE TESTED OUT OF SPECIFI...
1,NO DEVICE INTERROGATION POSSIBLE PT ALERT WAS ...
2,ICD TESTED OUT OF SPECIFICATIONS DURING MFGRS ...
3,DISSATISFIED WITH RESPONSEREPAIR LONG CHARGE T...
4,ALLEGED MALFUNCTIONDEFECTIVEBADFAILURE


In [9]:
# creating a list of device identifiers
listDevices = []



for n in df["STD_DEVICE"].value_counts().index:
    listDevices.append(n)

print(listDevices)

['StJUDE_DURATA', 'MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI', 'MEDTRONIC_PROTECTA_XT', 'MEDTRONIC_EVERA', 'MEDTRONIC_SPRINT_QUATTRO', 'STJUDE_FORTIFY_VR', 'StJUDE_ELLIPSE', 'MEDTRONIC_SPRINT', 'STJUDE_RIATA', 'BOS_SCI_EMBLEM', 'MEDTRONIC_VIVA', 'StJUDE_QUARTET', 'MEDTRONIC_TRANSVENE', 'STJUDE_RIATA_ST', 'MEDTRONIC_VISIA', 'MEDTRONIC_SPRINT_FIDELIS', 'BOS_SCI_INOGEN_EL', 'MEDTRONIC_GEM_DR', 'MEDTRONIC_COMPIA_MRI', 'MEDTRONIC_SUBQ', 'MEDTRONIC_MICRO_JEWEL', 'MEDTRONIC_AMPLIA_MRI', 'MEDTRONIC_BRAVA', 'MEDTRONIC_EGIDA', 'BOS_SCI_PERCIVA', 'StJUDE_GALLANT', 'STJUDE_TENDRIL_STS']


In [10]:
# replace device identifiers
# other versions of existing entries were added to account for typos
# spaces were not used in regexs as words may not be spaced
def cleanTextRound3(text):
    text = re.sub('''
                    EVERA|SPRINT QUATTRO|SPRINT|ELLIPSE|EMBLEM|VISIA|SUBQ|SUB Q|SUB-Q
                    ''', " DEVID ", text)
    return text

round3 = lambda x: cleanTextRound3(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round3))
data_clean.head(5)

,FOI_TEXT
0,APPARENT LEAD FRACTURE TESTED OUT OF SPECIFI...
1,NO DEVICE INTERROGATION POSSIBLE PT ALERT WAS ...
2,ICD TESTED OUT OF SPECIFICATIONS DURING MFGRS ...
3,DISSATISFIED WITH RESPONSEREPAIR LONG CHARGE T...
4,ALLEGED MALFUNCTIONDEFECTIVEBADFAILURE


In [11]:
# replace "EVENT DESCRIPTION" found at the start of many entries
def cleanTextRound4(text):
    text = re.sub("EVENT DESCRIPTION", "", text)
    return text

round4 = lambda x: cleanTextRound4(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round4))
data_clean.head(5)

,FOI_TEXT
0,APPARENT LEAD FRACTURE TESTED OUT OF SPECIFI...
1,NO DEVICE INTERROGATION POSSIBLE PT ALERT WAS ...
2,ICD TESTED OUT OF SPECIFICATIONS DURING MFGRS ...
3,DISSATISFIED WITH RESPONSEREPAIR LONG CHARGE T...
4,ALLEGED MALFUNCTIONDEFECTIVEBADFAILURE


In [12]:
# replace odd spacing
def cleanTextRound5(text):
    text = re.sub(" ", " ", text)
    text = re.sub("  ", " ", text)
    text = re.sub("   ", " ", text)
    text = re.sub("    ", " ", text)
    text = re.sub("     ", " ", text)
    return text

round5 = lambda x: cleanTextRound5(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round5))
data_clean.head(5)

,FOI_TEXT
0,APPARENT LEAD FRACTURE TESTED OUT OF SPECIFIC...
1,NO DEVICE INTERROGATION POSSIBLE PT ALERT WAS ...
2,ICD TESTED OUT OF SPECIFICATIONS DURING MFGRS ...
3,DISSATISFIED WITH RESPONSEREPAIR LONG CHARGE T...
4,ALLEGED MALFUNCTIONDEFECTIVEBADFAILURE


In [13]:
# remove stopwords
def cleanTextRound6(text):
    text = re.sub('''
                    I | ME | MY | MYSELF | WE | OUR | OURS | OURSELVES | YOU | YOUR | YOURS | YOURSELF 
                    | YOURSELVES | HE | HIM | HIS | HIMSELF | SHE | HER | HERS | HERSELF | IT | ITS | ITSELF 
                    | THEY | THEM | THEIR | THEIRS | THEMSELVES | WHAT | WHICH | WHO | WHOM | THIS | THAT | THESE 
                    | THOSE | AM | IS | ARE | WAS | WERE | BE | BEEN | BEING | HAVE | HAS | HAD | HAVING | DO 
                    | DOES | DID | DOING | A | AN | THE | AND | BUT | IF | OR | BECAUSE | AS | UNTIL | WHILE 
                    | OF | AT | BY | FOR | WITH | ABOUT | AGAINST | BETWEEN | INTO | THROUGH | DURING | BEFORE 
                    | AFTER | ABOVE | BELOW | TO | FROM | UP | DOWN | IN | OUT | ON | OFF | OVER | UNDER | AGAIN 
                    | FURTHER | THEN | ONCE | HERE | THERE | WHEN | WHERE | WHY | HOW | ALL | ANY | BOTH | EACH 
                    | FEW | MORE | MOST | OTHER | SOME | SUCH | NO | NOR | NOT | ONLY | OWN | SAME | SO | THAN 
                    | TOO | VERY | S | T | CAN | WILL | JUST | DON | SHOULD | NOW
                    ''', " ", text)
    return text

round6 = lambda x: cleanTextRound6(x)


data_clean = pd.DataFrame(data_clean.FOI_TEXT.apply(round6))
data_clean.head(5)

,FOI_TEXT
0,APPARENT LEAD FRACTURE TESTED OF SPECIFICATIO...
1,NO DEVICE INTERROGATION POSSIBLE PT ALERT ACTI...
2,ICD TESTED OF SPECIFICATIONS MFGRS ANALYSIS
3,DISSATISFIED RESPONSEREPAIR LONG CHARGE TIME E...
4,ALLEGED MALFUNCTIONDEFECTIVEBADFAILURE


In [14]:
# checking the first 100 reports in the df
n=0

for entry in data_clean["FOI_TEXT"]:
    if n < 100:
        print("[" + str(n) + "]\n" + entry + "\n")
        n += 1    

[0]
 APPARENT LEAD FRACTURE TESTED OF SPECIFICATIONS ANALYSIS

[1]
NO DEVICE INTERROGATION POSSIBLE PT ALERT ACTIVATED EXCESSIVE CHARGE TIME LEAD IMPEDANCE BATTERY VOLTAGE

[2]
ICD TESTED OF SPECIFICATIONS MFGRS ANALYSIS

[3]
DISSATISFIED RESPONSEREPAIR LONG CHARGE TIME ERI TACHY

[4]
ALLEGED MALFUNCTIONDEFECTIVEBADFAILURE

[5]
OVERSENSING LEAD FRACTURE CAUSED PT RECEIVE UNNECESSARY ICD DISCHARGES

[6]
ATTEMPTED IMPLANT TESTED OF SPECIFICATIONS MFGRS ANALYSIS

[7]
TESTED OF SPECIFICATIONS ANALYSIS

[8]
FRACTURED ELECTRODE COIL DEFIB COILPATCH

[9]
DEVICE INOPERABLE THERAPY ATTEMPTED DUE APPARENT FRACTURED NONMEDTRONIC LEAD TESTED OF SPECIFICATIONS ANALYSIS

[10]
TESTED OF SPECIFICATIOSN MFGRS ANALYSIS DEVICE NOTED BE OK EXPLANT RETURNED MONTHS FOLLOWING EXPLANT

[11]
 UNWANTEDUNEXPECTED SHOCK INTERFERENCENOISE INAPPROPRIATE THERAPY TESTED OF SPECIFICATIONS MFGRS ANALYSIS

[12]
TESTED OF SPECIFICATIONS MFGRS ANALYSIS

[13]
APPARENT LEAD FRACTURE  PATIENT REPORTS RECEIVING SHOCKS HIS LEA

In [15]:
# join the cleaned data with the main df
df["CLEAN_FOI_TEXT"] = data_clean
df["CLEAN_FOI_TEXT"].head()

0     APPARENT LEAD FRACTURE TESTED OF SPECIFICATIO...
1    NO DEVICE INTERROGATION POSSIBLE PT ALERT ACTI...
2          ICD TESTED OF SPECIFICATIONS MFGRS ANALYSIS
3    DISSATISFIED RESPONSEREPAIR LONG CHARGE TIME E...
4               ALLEGED MALFUNCTIONDEFECTIVEBADFAILURE
Name: CLEAN_FOI_TEXT, dtype: object

In [16]:
# stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def cleanTextRound7(text): 
    return " ".join([stemmer.stem(word) for word in text.split()])
df["CLEAN_FOI_TEXT"] = df["CLEAN_FOI_TEXT"].apply(cleanTextRound7)

In [17]:
# checking the first 100 reports in the df
n=0

for entry in df["CLEAN_FOI_TEXT"]:
    if n < 100:
        print("[" + str(n) + "]\n" + entry + "\n")
        n += 1    

[0]
appar lead fractur test of specif analysi

[1]
no devic interrog possibl pt alert activ excess charg time lead imped batteri voltag

[2]
icd test of specif mfgr analysi

[3]
dissatisfi responserepair long charg time eri tachi

[4]
alleg malfunctiondefectivebadfailur

[5]
oversens lead fractur caus pt receiv unnecessari icd discharg

[6]
attempt implant test of specif mfgr analysi

[7]
test of specif analysi

[8]
fractur electrod coil defib coilpatch

[9]
devic inoper therapi attempt due appar fractur nonmedtron lead test of specif analysi

[10]
test of specificatiosn mfgr analysi devic note be ok explant return month follow explant

[11]
unwantedunexpect shock interferencenois inappropri therapi test of specif mfgr analysi

[12]
test of specif mfgr analysi

[13]
appar lead fractur patient report receiv shock hi lead went bad short out

[14]
power reset inabl charg capacitor devic vibrat click attempt charg patient alert tone

[15]
power reset paramet telemetri problem preimplant te

In [18]:
df.shape

(91259, 41)

In [19]:
df.to_pickle("./4_MASDEV_Cleaned.pkl")

In [20]:
# creating dataframes for each set
df["STD_DEVICE"].value_counts()

StJUDE_DURATA                          26273
MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI    10459
MEDTRONIC_PROTECTA_XT                   9782
MEDTRONIC_EVERA                         6292
MEDTRONIC_SPRINT_QUATTRO                5617
STJUDE_FORTIFY_VR                       5038
StJUDE_ELLIPSE                          4940
MEDTRONIC_SPRINT                        3950
STJUDE_RIATA                            3538
BOS_SCI_EMBLEM                          3522
MEDTRONIC_VIVA                          2457
StJUDE_QUARTET                          2029
MEDTRONIC_TRANSVENE                     1676
STJUDE_RIATA_ST                         1184
MEDTRONIC_VISIA                         1057
MEDTRONIC_SPRINT_FIDELIS                 998
BOS_SCI_INOGEN_EL                        533
MEDTRONIC_GEM_DR                         402
MEDTRONIC_COMPIA_MRI                     298
MEDTRONIC_SUBQ                           293
MEDTRONIC_MICRO_JEWEL                    278
MEDTRONIC_AMPLIA_MRI                     234
MEDTRONIC_

In [21]:
# set 1

In [22]:
dfSet1 = df[df["STD_DEVICE"]=="BOS_SCI_EMBLEM"]
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="StJUDE_ELLIPSE"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="STJUDE_RIATA"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="STJUDE_RIATA_ST"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="STJUDE_FORTIFY_VR"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="MEDTRONIC_SPRINT"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="MEDTRONIC_SPRINT_QUATTRO"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="STJUDE_TENDRIL_STS"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="BOS_SCI_INOGEN_EL"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="BOS_SCI_PERCIVA"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="MEDTRONIC_EVERA"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="StJUDE_DURATA"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="StJUDE_GALLANT"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="StJUDE_QUARTET"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="MEDTRONIC_GEM_DR"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="MEDTRONIC_BRAVA"])
dfSet1 = dfSet1.append(df[df["STD_DEVICE"]=="MEDTRONIC_VIVA"])

In [23]:
dfSet1.shape

(76513, 41)

In [24]:
dfSet1["STD_DEVICE"].value_counts()

StJUDE_DURATA                          26273
MEDTRONIC_SPRINT_QUATTRO_SECURE_MRI    10459
MEDTRONIC_EVERA                         6292
MEDTRONIC_SPRINT_QUATTRO                5617
STJUDE_FORTIFY_VR                       5038
StJUDE_ELLIPSE                          4940
MEDTRONIC_SPRINT                        3950
STJUDE_RIATA                            3538
BOS_SCI_EMBLEM                          3522
MEDTRONIC_VIVA                          2457
StJUDE_QUARTET                          2029
STJUDE_RIATA_ST                         1184
BOS_SCI_INOGEN_EL                        533
MEDTRONIC_GEM_DR                         402
MEDTRONIC_BRAVA                          207
BOS_SCI_PERCIVA                           39
StJUDE_GALLANT                            26
STJUDE_TENDRIL_STS                         7
Name: STD_DEVICE, dtype: int64

In [25]:
dfSet1.to_pickle("./4_MASDEV_Cleaned_S1.pkl")

In [26]:
# set 2

In [31]:
dfSet2 = df[df["STD_DEVICE"]=="MEDTRONIC_VISIA"]
dfSet2 = dfSet2.append(df[df["STD_DEVICE"]=="MEDTRONIC_SPRINT_FIDELIS"])
dfSet2 = dfSet2.append(df[df["STD_DEVICE"]=="MEDTRONIC_MICRO_JEWEL"])
dfSet2 = dfSet2.append(df[df["STD_DEVICE"]=="MEDTRONIC_SUBQ"])
#dfSet2 = dfSet2.append(df[df["STD_DEVICE"]=="MEDTRONIC_PROTECTA_XT"])
dfSet2 = dfSet2.append(df[df["STD_DEVICE"]=="MEDTRONIC_TRANSVENE"])
#dfSet2 = dfSet2.append(df[df["STD_DEVICE"]=="MEDTRONIC_COMPIA_MRI"])
#dfSet2 = dfSet2.append(df[df["STD_DEVICE"]=="MEDTRONIC_AMPLIA_MRI"])
dfSet2 = dfSet2.append(df[df["STD_DEVICE"]=="MEDTRONIC_EGIDA"])

In [32]:
dfSet2.shape

(4432, 41)

In [33]:
dfSet2["STD_DEVICE"].value_counts()

MEDTRONIC_TRANSVENE         1676
MEDTRONIC_VISIA             1057
MEDTRONIC_SPRINT_FIDELIS     998
MEDTRONIC_SUBQ               293
MEDTRONIC_MICRO_JEWEL        278
MEDTRONIC_EGIDA              130
Name: STD_DEVICE, dtype: int64

In [35]:
dfSet2.to_pickle("./4_MASDEV_Cleaned_S2.pkl")